In [ ]:
#| default_exp theta

In [ ]:
#| export
import math

import numpy as np
from scipy.stats import norm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

from statsforecast._lib import theta as _theta
from statsforecast.arima import is_constant
from statsforecast.utils import _repeat_val_seas, _seasonal_naive, results

# Theta Model

In [ ]:
#| hide
from statsforecast.utils import AirPassengers as ap

In [ ]:
#| hide
initial_smoothed = ap[0] / 2
alpha = 0.5
theta = 2
_theta.init_state(ap, _theta.ModelType.STM, initial_smoothed, alpha, theta)
_theta.init_state(ap, _theta.ModelType.OTM, initial_smoothed, alpha, theta)
_theta.init_state(ap, _theta.ModelType.DSTM, initial_smoothed, alpha, theta)
_theta.init_state(ap, _theta.ModelType.DOTM, initial_smoothed, alpha, theta)

In [ ]:
#| hide
#simple theta model tests
nmse_ = len(ap)
amse_ = np.zeros(30)
e_ = np.zeros(len(ap))
initial_smoothed = ap[0] / 2
alpha = 0.5
theta = 2.
init_states = np.zeros((len(ap), 5))
mse = _theta.calc(
    ap,
    init_states, 
    _theta.ModelType.STM, 
    initial_smoothed,
    alpha,
    theta,
    e_,
    amse_,
    3,
)
#verify we recover the fitted values
np.testing.assert_array_equal(
    ap - e_,
    init_states[:, -1]
)

#verify we get same fitted values than R
# use stm(AirPassengers, s=F, estimation=F, h = 12)
# to recover
np.testing.assert_array_almost_equal(
    init_states[:, -1][[0, 1, -1]],
    np.array([101.1550, 107.9061, 449.1692]), 
    decimal=2
)

# recover mse
assert math.isclose(np.sum(e_[3:] ** 2) / np.mean(np.abs(ap)), mse)

# test forecasts
h = 5
fcsts = np.zeros(h)
_theta.forecast(
    init_states,
    len(ap), 
    _theta.ModelType.STM, 
    fcsts,
    alpha,
    theta,
)
# test same forecast than R's
np.testing.assert_allclose(
    fcsts,
    np.array([441.9132, 443.2418, 444.5704, 445.8990, 447.2276]),
)

In [ ]:
#| hide
#optimal theta model tests
nmse_ = len(ap)
amse_ = np.zeros(30)
e_ = np.zeros(len(ap))
initial_smoothed = ap[0] / 2
alpha = 0.5
theta = 2.
init_states = np.zeros((len(ap), 5))
mse = _theta.calc(
    ap,
    init_states, 
    _theta.ModelType.OTM, 
    initial_smoothed,
    alpha,
    theta,
    e_,
    amse_,
    3,
)
#verify we recover the fitted values
np.testing.assert_array_equal(
    ap - e_,
    init_states[:, -1]
)
#verify we get same fitted values than R
# use stm(AirPassengers, s=F, estimation=F, h = 12)
# to recover
np.testing.assert_allclose(
    init_states[:, -1][[0, 1, -1]],
    np.array([101.1550, 107.9061, 449.1692]),
    rtol=1e-5,
)
# recover mse
assert math.isclose(np.sum(e_[3:] ** 2) / np.mean(np.abs(ap)), mse)

# test forecasts
h = 5
fcsts = np.zeros(h)
_theta.forecast(
    init_states,
    len(ap), 
    _theta.ModelType.OTM, 
    fcsts,
    alpha,
    theta,
)
# test same forecast than R's
np.testing.assert_allclose(
    fcsts,
    np.array([441.9132, 443.2418, 444.5704, 445.8990, 447.2276]),
)

In [ ]:
#| hide
#dynamic simple theta model tests
nmse_ = len(ap)
amse_ = np.zeros(30)
e_ = np.zeros(len(ap))
initial_smoothed = ap[0] / 2
alpha = 0.5
theta = 2.
init_states = np.zeros((len(ap), 5))
mse = _theta.calc(
    ap,
    init_states, 
    _theta.ModelType.DSTM, 
    initial_smoothed,
    alpha,
    theta,
    e_,
    amse_,
    3,
)
#verify we recover the fitted values
np.testing.assert_allclose(
    ap - e_,
    init_states[:, -1]
)
#verify we get same fitted values than R
# use dstm(AirPassengers, s=F, estimation=F, h = 12)
# to recover
np.testing.assert_allclose(
    init_states[:, -1][[0, 1, -1]],
    np.array([112.0000, 112.0000, 449.1805]),
    rtol=1e-5,
)
# recover mse
assert math.isclose(np.sum(e_[3:] ** 2) / np.mean(np.abs(ap)), mse)

# test forecasts
h = 5
fcsts = np.zeros(h)
_theta.forecast(
    init_states,
    len(ap), 
    _theta.ModelType.DSTM, 
    fcsts,
    alpha,
    theta
)
# test same forecast than R's
np.testing.assert_allclose(
    fcsts,
    np.array([441.9132, 443.2330, 444.5484, 445.8594, 447.1659]),
)

In [ ]:
#| hide
#dynamic optimal theta model tests
nmse_ = len(ap)
amse_ = np.zeros(30)
e_ = np.zeros(len(ap))
initial_smoothed = ap[0] / 2
alpha = 0.5
theta = 2.
init_states = np.zeros((len(ap), 5))
mse = _theta.calc(
    ap,
    init_states, 
    _theta.ModelType.DOTM, 
    initial_smoothed,
    alpha,
    theta,
    e_,
    amse_,
    3
)
#verify we recover the fitted values
np.testing.assert_allclose(
    ap - e_,
    init_states[:, -1]
)
#verify we get same fitted values than R
# use dstm(AirPassengers, s=F, estimation=F, h = 12)
# to recover
np.testing.assert_allclose(
    init_states[:, -1][[0, 1, -1]],
    np.array([112.0000, 112.0000, 449.1805]),
    rtol=1e-5,
)
# recover mse
assert math.isclose(np.sum(e_[3:] ** 2) / np.mean(np.abs(ap)), mse)

# test forecasts
h = 5
fcsts = np.zeros(h)
_theta.forecast(
    init_states,
    len(ap), 
    _theta.ModelType.DOTM, 
    fcsts,
    alpha,
    theta
)
# test same forecast than R's
np.testing.assert_allclose(
    fcsts,
    np.array([441.9132, 443.2330, 444.5484, 445.8594, 447.1659]),
)

In [ ]:
#| exporti
def initparamtheta(
    initial_smoothed: float,
    alpha: float,
    theta: float,
    y: np.ndarray,
    modeltype: _theta.ModelType,
):
    if modeltype in [_theta.ModelType.STM, _theta.ModelType.DSTM]:
        if math.isnan(initial_smoothed):
            initial_smoothed = y[0] / 2
            optimize_level = True
        else:
            optimize_level = False
        if math.isnan(alpha):
            alpha = 0.5
            optimize_alpha = True
        else:
            optimize_alpha = False
        theta = 2.0 # no optimize
        optimize_theta = False
    else:
        if math.isnan(initial_smoothed):
            initial_smoothed = y[0] / 2
            optimize_level = True
        else:
            optimize_level = False
        if math.isnan(alpha):
            alpha = 0.5
            optimize_alpha = True
        else:
            optimize_alpha = False
        if math.isnan(theta):
            theta = 2.0
            optimize_theta = True
        else:
            optimize_theta = False
    return {
        'initial_smoothed': initial_smoothed,
        'optimize_initial_smoothed': optimize_level,
        'alpha': alpha,
        'optimize_alpha': optimize_alpha,
        'theta': theta,
        'optimize_theta': optimize_theta,
    }

In [ ]:
#| hide
initparamtheta(
    initial_smoothed=np.nan,
    alpha=np.nan,
    theta=np.nan,
    y=ap,
    modeltype=_theta.ModelType.DOTM,
)

In [ ]:
#| exporti
def switch_theta(model: str) -> _theta.ModelType:
    if model == 'STM':
        return _theta.ModelType.STM
    if model == 'OTM':
        return _theta.ModelType.OTM
    if model == 'DSTM':
        return _theta.ModelType.DSTM
    if model == 'DOTM':
        return _theta.ModelType.DOTM
    raise ValueError(f'Invalid model type: {model}.')

In [ ]:
#| hide
switch_theta('STM')

In [ ]:
#| exporti
def optimize_theta_target_fn(
    init_par: dict[str, float],
    optimize_params: dict[str, bool],
    y: np.ndarray, 
    modeltype: _theta.ModelType,
    nmse: int,
):
    x0 = []
    lower = []
    upper = []
    lower_bounds = {
        'initial_smoothed': -1e10,
        'alpha': 0.1,
        'theta': 1.0,
    }
    upper_bounds = {
        'initial_smoothed': 1e10,
        'alpha': 0.99,
        'theta': 1e10,
    }
    for param, optim in optimize_params.items():
        if optim:
            x0.append(init_par[param])
            lower.append(lower_bounds[param])
            upper.append(upper_bounds[param])
    if not x0:
        return
    x0 = np.array(x0)
    lower = np.array(lower)
    upper = np.array(upper)

    init_level = init_par['initial_smoothed']
    init_alpha = init_par['alpha']
    init_theta = init_par['theta']
    
    opt_level = optimize_params['initial_smoothed']
    opt_alpha = optimize_params['alpha']
    opt_theta = optimize_params['theta']

    opt_res = _theta.optimize(
        x0,
        lower,
        upper,
        init_level,
        init_alpha,
        init_theta,
        opt_level,
        opt_alpha,
        opt_theta,
        y,
        modeltype,
        nmse
    )
    return results(*opt_res, None)  # type: ignore

In [ ]:
#| exporti
def thetamodel(
    y: np.ndarray,
    m: int, 
    modeltype: str, 
    initial_smoothed: float,
    alpha: float,
    theta: float,
    nmse: int
):
    model_type = switch_theta(modeltype)
    #initial parameters
    par = initparamtheta(
        initial_smoothed=initial_smoothed, 
        alpha=alpha,
        theta=theta, 
        y=y,
        modeltype=model_type,
    )
    optimize_params = {key.replace('optimize_', ''): val for key, val in par.items() if 'optim' in key}
    par = {key: val for key, val in par.items() if 'optim' not in key}
    # parameter optimization
    fred = optimize_theta_target_fn(
        init_par=par,
        optimize_params=optimize_params,
        y=y, 
        modeltype=model_type,
        nmse=nmse,
    )
    if fred is not None:
        fit_par = fred.x
    j = 0
    if optimize_params['initial_smoothed']:
        par['initial_smoothed'] = fit_par[j]
        j += 1
    if optimize_params['alpha']:
        par['alpha'] = fit_par[j]
        j += 1
    if optimize_params['theta']:
        par['theta'] = fit_par[j]
        j += 1
    
    amse, e, states, mse = _theta.pegels_resid(
        y,
        model_type,
        par['initial_smoothed'],
        par['alpha'],
        par['theta'],
        nmse,
    )

    return dict(mse=mse, amse=amse, fit=fred, residuals=e,
                m=m, states=states, par=par, n=len(y), 
                modeltype=modeltype, mean_y=np.mean(y))

In [ ]:
#| hide
res = thetamodel(
    y=ap,
    m=12,
    modeltype='STM',
    initial_smoothed=np.nan,
    alpha=np.nan,
    theta=np.nan, 
    nmse=3
)

In [ ]:
#| exporti
def compute_pi_samples(n, h, states, sigma, alpha, theta, mean_y, seed=0, n_samples=200):
    samples = np.full((h, n_samples), fill_value=np.nan, dtype=np.float32)
    # states: level, meany, An, Bn, mu
    smoothed, _, A, B, _ = states[-1]
    np.random.seed(seed)
    for i in range(n, n + h):
        samples[i - n] = smoothed + (1 - 1 / theta)*(A*((1 - alpha) ** i) + B * (1 - (1 - alpha)**(i + 1)) / alpha)
        samples[i - n] += np.random.normal(scale=sigma, size=n_samples)
        smoothed = alpha * samples[i - n] + (1 - alpha) * smoothed
        mean_y = (i * mean_y + samples[i - n]) / (i + 1)
        B = ((i - 1) * B + 6 * (samples[i - n] - mean_y) / (i + 1)) / (i + 2)
        A = mean_y - B * (i + 2) / 2
    return samples

In [ ]:
#| export
def forecast_theta(obj, h, level=None):
    forecast = np.empty(h)
    n = obj['n']
    states = obj['states']
    alpha=obj['par']['alpha']
    theta=obj['par']['theta']
    _theta.forecast(
        states,
        n,
        switch_theta(obj['modeltype']), 
        forecast,
        alpha,
        theta,
    )
    res = {'mean': forecast}

    if level is not None:
        sigma = np.std(obj['residuals'][3:], ddof=1)
        mean_y = obj['mean_y']
        samples = compute_pi_samples(n=n, h=h, states=states, sigma=sigma, alpha=alpha, 
                                     theta=theta, mean_y=mean_y)
        for lv in level:
            min_q = (100 - lv) / 200
            max_q = min_q + lv / 100
            res[f'lo-{lv}'] = np.quantile(samples, min_q, axis=1)
            res[f'hi-{lv}'] = np.quantile(samples, max_q, axis=1)

    if obj.get('decompose', False):
        seas_forecast = _repeat_val_seas(obj['seas_forecast']['mean'], h=h)
        for key in res:
            if obj['decomposition_type'] == 'multiplicative':
                res[key] = res[key] * seas_forecast
            else:
                res[key] = res[key] + seas_forecast
    return res

In [ ]:
forecast_theta(res, 12, level=[90, 80])

In [ ]:
#| export
def auto_theta(
    y,
    m,
    model=None, 
    initial_smoothed=None,
    alpha=None, 
    theta=None,
    nmse=3,
    decomposition_type='multiplicative'
):
    # converting params to floats 
    # to improve numba compilation
    if initial_smoothed is None:
        initial_smoothed = np.nan
    if alpha is None:
        alpha = np.nan
    if theta is None:
        theta = np.nan
    if nmse < 1 or nmse > 30:
        raise ValueError('nmse out of range')
    # constan values
    if is_constant(y):
        thetamodel(y=y, m=m, modeltype='STM', nmse=nmse, 
                  initial_smoothed=np.mean(y) / 2, alpha=0.5, theta=2.0)
    # seasonal decomposition if needed
    decompose = False
    # seasonal test
    if m >= 4 and len(y) >= 2 * m:
        r = acf(y, nlags=m, fft=False)[1:]
        stat = np.sqrt((1 + 2 * np.sum(r[:-1]**2)) / len(y))
        decompose = np.abs(r[-1]) / stat > norm.ppf(0.95)

    data_positive = min(y) > 0
    if decompose:
        # change decomposition type if data is not positive
        if decomposition_type == 'multiplicative' and not data_positive:
            decomposition_type = 'additive'
        y_decompose = seasonal_decompose(y, model=decomposition_type, period=m).seasonal
        if decomposition_type == 'multiplicative' and any(y_decompose < 0.01):
            decomposition_type = 'additive'
            y_decompose = seasonal_decompose(y, model='additive', period=m).seasonal
        if decomposition_type == 'additive':
            y = y - y_decompose
        else:
            y = y / y_decompose
        seas_forecast = _seasonal_naive(y=y_decompose, h=m, season_length=m, fitted=False)
    
    # validate model
    if model not in [None, 'STM', 'OTM', 'DSTM', 'DOTM']:
        raise ValueError(f'Invalid model type: {model}.')

    n = len(y)
    npars = 3 
    #non-optimized tiny datasets
    if n <= npars:
        raise NotImplementedError('tiny datasets')
    if model is None:
        modeltype = ['STM', 'OTM', 'DSTM', 'DOTM']
    else:
        modeltype = [model]
        
    best_ic = np.inf
    for mtype in modeltype:
        fit = thetamodel(y=y, m=m, modeltype=mtype, nmse=nmse, 
                         initial_smoothed=initial_smoothed, alpha=alpha, theta=theta)
        fit_ic = fit['mse']
        if not np.isnan(fit_ic):
            if fit_ic < best_ic:
                model = fit
                best_ic = fit_ic
    if np.isinf(best_ic):
        raise Exception('no model able to be fitted')

    if decompose:
        if decomposition_type == 'multiplicative':
            model['residuals'] = model['residuals'] * y_decompose
        else:
            model['residuals'] = model['residuals'] + y_decompose
        model['decompose'] = decompose
        model['decomposition_type'] = decomposition_type
        model['seas_forecast'] = dict(seas_forecast)
    return model

In [ ]:
#| hide
# test zero constant time series
zeros = np.zeros(30, dtype=np.float32)
res = auto_theta(zeros, m=12)
forecast_theta(res, 28)

In [ ]:
#| hide
import matplotlib.pyplot as plt

In [ ]:
#| hide
res = auto_theta(ap, m=12)
fcst = forecast_theta(res, 12, level=[80, 90])
plt.plot(np.arange(0, len(ap)), ap)
plt.plot(np.arange(len(ap), len(ap) + 12), fcst['mean'])
plt.fill_between(np.arange(len(ap), len(ap) + 12), 
                 fcst['lo-90'], 
                 fcst['hi-90'], 
                 color='orange')

In [ ]:
#| hide
res = auto_theta(ap, m=12, model='DOTM', decomposition_type='additive')
fcst = forecast_theta(res, 12, level=[80, 90])
plt.plot(np.arange(0, len(ap)), ap)
plt.plot(np.arange(len(ap), len(ap) + 12), fcst['mean'])
plt.fill_between(np.arange(len(ap), len(ap) + 12), 
                 fcst['lo-90'], 
                 fcst['hi-90'], 
                 color='orange')

In [ ]:
#| hide
# test Simple Theta Model
# with no seasonality
res = auto_theta(ap, m=1, model='STM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([432.9292, 434.2578, 435.5864, 436.9150, 438.2435]),
    fcst['mean'],
    rtol=1e-5,
)

# test Simple Theta Model
# with seasonality
res = auto_theta(ap, m=12, model='STM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([440.7886, 429.0739, 490.4933, 476.4663, 480.4363]),
    fcst['mean'],
    rtol=1e-3,
)

In [ ]:
#| hide
# test Optimized Theta Model
# with no seasonality
res = auto_theta(ap, m=1, model='OTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([433.3307, 435.0567, 436.7828, 438.5089, 440.2350]),
    fcst['mean'],
    rtol=0.01,
)

# test Optimized Theta Model
# with seasonality
res = auto_theta(ap, m=12, model='OTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([442.8492, 432.1255, 495.1706, 482.1585, 487.3280]),
    fcst['mean'],
    rtol=1e-3,
)

In [ ]:
#| hide
# test Dynamic Simple Theta Model
# with no seasonality
res = auto_theta(ap, m=1, model='DSTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([432.9292, 434.2520, 435.5693, 436.8809, 438.1871]),
    fcst['mean'],
    rtol=1e-4,
)

# test Simple Theta Model
# with seasonality
res = auto_theta(ap, m=12, model='DSTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([440.7631, 429.0512, 490.4711, 476.4495, 480.4251]),
    fcst['mean'],
    rtol=1e-4,
)

In [ ]:
#| hide
# test Dynamic Optimized Theta Model
# with no seasonality
res = auto_theta(ap, m=1, model='DOTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([432.5131, 433.4257, 434.3344, 435.2391, 436.1399]),
    fcst['mean'],
    rtol=0.01,
)

# test Simple Theta Model
# with seasonality
res = auto_theta(ap, m=12, model='DOTM')
fcst = forecast_theta(res, 5)
np.testing.assert_allclose(
    np.array([442.9720, 432.3586, 495.5702, 482.6789, 487.9888]),
    fcst['mean'],
    rtol=0.01,
)

In [ ]:
#| hide
# test inttermitent time series
inttermitent_series = np.array([
    1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 3., 0., 1., 0., 0., 0., 0.,
    0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 3., 0., 0., 0., 0., 0., 0.,
    0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 1., 1.,
    0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
    0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
    0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 3., 1., 0., 1., 0., 0., 0.,
    1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2.,
    1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 1., 2., 0.,
    1., 0., 2., 2., 0., 0., 1., 2., 0., 0., 0., 2., 0., 1., 0., 0., 0.,
    0., 2., 0., 1., 0., 2., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
    0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1., 1., 0.,
    0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 2.,
    1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2., 0., 1., 0.,
    0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
    1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 2., 0., 0., 0., 0., 1., 0., 1., 0., 2., 0., 0., 2., 0., 0.,
    2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2.,
    0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
    1., 0., 1., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
    0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 0., 0., 2., 0., 0., 1., 0., 2., 0., 0., 0., 0.,
    2., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
    1., 0., 1., 0., 0., 0., 0., 3., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
    0., 0., 0., 2., 0., 1., 0., 2., 1., 2., 2., 0., 0., 0., 0., 0., 0.,
    0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 2., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
    0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 2., 2.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 4., 0., 0., 0., 0., 0., 1.,
    1., 0., 0., 1., 1., 0., 0., 2., 1., 1., 1., 2., 1., 0., 0., 0., 1.,
    0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
    0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
    0., 0., 0., 0., 0., 1., 2., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
    1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
    1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 2., 0., 0.,
    1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
    0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
    0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
    0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
    0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
    1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
    0., 0., 0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 2., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 1.,
    0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
    1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
    1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
    1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
    0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
    1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
    0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
    0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
    0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
    0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
    0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
    0., 0., 3., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
    0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 1.,
    0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
    0., 0., 2., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

for season_length in [1, 7]:
    res = auto_theta(inttermitent_series, m=season_length)
    fcst = forecast_theta(res, 28)
    plt.plot(np.arange(0, len(inttermitent_series)), inttermitent_series)
    plt.plot(np.arange(len(inttermitent_series), len(inttermitent_series) + 28), fcst['mean'])
    plt.show()

In [ ]:
#| export
def forward_theta(fitted_model, y):
    m = fitted_model['m']
    model = fitted_model['modeltype']
    initial_smoothed = fitted_model['par']['initial_smoothed']
    alpha = fitted_model['par']['alpha']
    theta = fitted_model['par']['theta']
    return auto_theta(y=y, m=m, model=model, 
                      initial_smoothed=initial_smoothed, 
                      alpha=alpha, 
                      theta=theta)

In [ ]:
#| hide
res = auto_theta(ap, m=12)
np.testing.assert_allclose(
    forecast_theta(forward_theta(res, ap), h=12)['mean'],
    forecast_theta(res, h=12)['mean']
)
# test tranfer
forecast_theta(forward_theta(res, inttermitent_series), h=12, level=[80,90])
res_transfer = forward_theta(res, inttermitent_series)
for key in res_transfer['par']:
    assert res['par'][key] == res_transfer['par'][key]